# Il calcolo del pi-greco

The value of PI can be calculated in various ways. Consider the Monte Carlo method of approximating PI:

Incriviamo una circonferenza $C$ di raggio $r$ all'interno di un quadrato di lato $2r$. L'area del cerchio sara' $\pi r^2 $ e l'area del quadrato $4r^2$ e il loro rapporto $\pi r^2 / 4r^2 = \pi/4 $.

Se generiamo $N$ punti random nel quadrato, circa $M = N \pi/4$ dovrebbero cadere all'interno della circonferenza. Allora possiamo approssimare $\pi$ come
$$ \pi = 4*M/N. $$
Notiamo che piu' numeri random generiamo, piu' accurata sara' la stima.

Serial pseudo code for this procedure: 

![](https://computing.llnl.gov/tutorials/parallel_comp/images/pi1.gif)

The problem is computationally intensive - most of the time is spent executing the loop


-    Il problema si puo' parallelizzare?
-    Come dovrebbe essere partizionato il problema? 
-    Sono necessarie comunicazioni tra threads? 
-    Ci sono dipendenze tra i dati?
-    Sono necessarie sincronizzazioni?
-    Il load balancing puo' essere un problema? 

https://computing.llnl.gov/tutorials/parallel_comp/


###################################################

Attenzione!~ Numba non si appogggia a cuRAND, il Random Number Generator (RNG) si numba e' un'implementazione dell'algoritmo xoroshiro128+ (http://xoshiro.di.unimi.it/).

In [3]:
import numpy as np
import random
from numba import cuda
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32

In [1]:
@cuda.jit
def compute_pi(rng_states, iterations, out):
    """Find the maximum value in values and store in result[0]"""
    thread_id = cuda.grid(1)

    # Compute pi by drawing random (x, y) points and finding what
    # fraction lie inside a unit circle
    inside = 0
    for i in range(iterations):
        # Initialize RNG states on the GPU for parallel generators.
        x = xoroshiro128p_uniform_float32(rng_states, thread_id)
        y = xoroshiro128p_uniform_float32(rng_states, thread_id)
        if x**2 + y**2 <= 1.0:
            inside += 1

    out[thread_id] = 4.0 * inside / iterations

pi: 3.1416733


In [5]:
npoints = 100000000
circle_count = 0
inside = 0

for i in range(npoints):
    #generate 2 random numbers between 0 and 1
    x = random.random()
    y = random.random()
    if x**2 + y**2 <= 1.0:
        inside += 1
#endfor

print('pi =', 4.0*inside/npoints)

pi = 3.14134852


In [6]:
threads_per_block = 64
blocks = 24

#Returns a new device array initialized for n random number generators.
rng_states = create_xoroshiro128p_states(threads_per_block * blocks, seed=1)
out = np.zeros(threads_per_block * blocks, dtype=np.float32)

compute_pi[blocks, threads_per_block](rng_states, npoints, out)
print('pi:', out.mean())

pi: 3.1415894
